In [1]:
import pandas as pd
import scipy.stats as stat
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import Imputer
import math
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures

In [8]:
train_data = pd.read_csv('dengue_features_train.csv')
test_data = pd.read_csv('dengue_features_test.csv')
labels_train = pd.read_csv('dengue_labels_train.csv')
submission_data = pd.read_csv('submission_format.csv')

In [9]:
data = pd.concat([train_data, labels_train], axis=1, join_axes=[train_data.index])

In [10]:
data.head(5)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,city,year,weekofyear,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,2.628571,25.442857,6.900000,29.4,20.0,16.0,sj,1990,18,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,2.371429,26.714286,6.371429,31.7,22.2,8.6,sj,1990,19,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,2.300000,26.714286,6.485714,32.2,22.8,41.4,sj,1990,20,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,2.428571,27.471429,6.771429,33.3,23.3,4.0,sj,1990,21,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,3.014286,28.942857,9.371429,35.0,23.9,5.8,sj,1990,22,6


In [11]:
_,i = np.unique(data.columns, return_index=True)
data = data.iloc[:,i]
data = data.dropna()
data.head(5)

,city,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,week_start_date,weekofyear,year
0,sj,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,...,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,1990-04-30,18,1990
1,sj,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,...,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,1990-05-07,19,1990
2,sj,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,...,2.300000,26.714286,6.485714,32.2,22.8,41.4,4,1990-05-14,20,1990
3,sj,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,...,2.428571,27.471429,6.771429,33.3,23.3,4.0,3,1990-05-21,21,1990
4,sj,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,...,3.014286,28.942857,9.371429,35.0,23.9,5.8,6,1990-05-28,22,1990


In [12]:
data['week_value'] = data.year *100 + data.weekofyear
test_data['week_value'] = test_data.year *100 + test_data.weekofyear

In [14]:
test_data.head(5)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,week_value
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2,200818
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3,200819
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0,200820
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3,200821
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1,200822


In [18]:
data_sj = data[data['city']=='sj']
data_iq = data[data['city']=='iq']

test_data_sj = test_data[test_data['city']=='sj']
test_data_iq = test_data[test_data['city']=='iq']

In [19]:
total_cases_sj = data_sj[['total_cases']].values
total_cases_iq = data_iq[['total_cases']].values

In [30]:
features = ['week_value','year','weekofyear','ndvi_ne','ndvi_nw','ndvi_se','ndvi_sw','precipitation_amt_mm','reanalysis_air_temp_k','reanalysis_avg_temp_k','reanalysis_dew_point_temp_k','reanalysis_max_air_temp_k','reanalysis_min_air_temp_k','reanalysis_precip_amt_kg_per_m2','reanalysis_relative_humidity_percent','reanalysis_sat_precip_amt_mm','reanalysis_specific_humidity_g_per_kg','reanalysis_tdtr_k','station_avg_temp_c','station_diur_temp_rng_c','station_max_temp_c','station_min_temp_c','station_precip_mm']

selected_features = []
non_selected_features = []
r_list =[]
f_list=[]
for f in features: 
    r = stat.pearsonr(data_sj[['total_cases']],data_sj[[f]])
    if (r[0]>0.1):
        selected_features.append(f)
    else:
        r_list.append(r[0])
        non_selected_features.append(f)
       

In [32]:
non_selected_features


['week_value',
 'year',
 'ndvi_ne',
 'ndvi_nw',
 'ndvi_se',
 'ndvi_sw',
 'reanalysis_tdtr_k',
 'station_diur_temp_rng_c',
 'station_precip_mm']

In [34]:
data_sj

,city,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,week_start_date,weekofyear,year,week_value
0,sj,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,...,25.442857,6.900000,29.4,20.0,16.0,4,1990-04-30,18,1990,199018
1,sj,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,...,26.714286,6.371429,31.7,22.2,8.6,5,1990-05-07,19,1990,199019
2,sj,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,...,26.714286,6.485714,32.2,22.8,41.4,4,1990-05-14,20,1990,199020
3,sj,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,...,27.471429,6.771429,33.3,23.3,4.0,3,1990-05-21,21,1990,199021
4,sj,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,...,28.942857,9.371429,35.0,23.9,5.8,6,1990-05-28,22,1990,199022
6,sj,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,299.221429,295.865714,301.3,...,27.414286,6.771429,32.2,23.3,29.7,4,1990-06-11,24,1990,199024
7,sj,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,299.528571,296.531429,300.6,...,28.371429,7.685714,33.9,22.8,21.1,5,1990-06-18,25,1990,199025
8,sj,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,299.557143,296.378571,302.1,...,28.328571,7.385714,33.9,22.8,21.1,10,1990-06-25,26,1990,199026
10,sj,0.192875,0.082350,0.191943,0.152929,22.27,299.512857,299.592857,296.041429,301.8,...,27.557143,7.157143,31.7,21.7,63.7,8,1990-07-09,28,1990,199028
11,sj,0.291600,0.211800,0.301200,0.280667,59.17,299.667143,299.750000,296.334286,302.0,...,28.128571,6.900000,32.8,23.9,12.2,2,1990-07-16,29,1990,199029


In [ ]:
#data = data.replace('sj',1)
#data = data.replace('iq',2)
#test_data = test_data.replace('sj',1)
#test_data = test_data.replace('iq',2)

#selected_f = ['year','reanalysis_air_temp_k','reanalysis_min_air_temp_k','reanalysis_tdtr_k','station_diur_temp_rng_c','station_min_temp_c']
#selected_f_sj = ['week_value','weekofyear','reanalysis_dew_point_temp_k','reanalysis_max_air_temp_k','reanalysis_specific_humidity_g_per_kg']

selected_f = features;
selected_f_sj = features;

poly_sj = PolynomialFeatures(degree=3);
poly_data_sj = poly_sj.fit_transform(data_sj[selected_f_sj]);

x= data_sj['total_cases'];

linreg_sj = LinearRegression()
linreg_sj.fit(poly_data_sj,data_sj['total_cases'])
train_pred_sj = linreg_sj.predict(poly_data_sj)

ceil_train_pred_sj =[]
round_train_pred_sj = []
for d in train_pred_sj:
    if(d<0):
        d=0

    else:
        d=math.ceil(d)
        dr = round(d)
    ceil_train_pred_sj.append(d)
    round_train_pred_sj.append(d)

sqrt_sj = np.sqrt(metrics.mean_squared_error(data_sj[['total_cases']],ceil_train_pred_sj))
print(mean_absolute_error(data_sj[['total_cases']],ceil_train_pred_sj))
print(mean_absolute_error(data_sj[['total_cases']],round_train_pred_sj))
print(sqrt_sj)

poly_iq = PolynomialFeatures(degree=3);
poly_data_iq = poly_iq.fit_transform(data_iq[selected_f]);

linreg_iq = LinearRegression()
linreg_iq.fit(poly_data_iq,data_iq['total_cases'])
train_pred_iq = linreg_iq.predict(poly_data_iq)

ceil_train_pred_iq =[]
round_train_pred_iq = []
for d in train_pred_iq:
    if(d<0):
        d=0

    else:
        d=math.ceil(d)
        dr = round(d)
    ceil_train_pred_iq.append(d)
    round_train_pred_iq.append(dr)

sqrt_iq = np.sqrt(metrics.mean_squared_error(data_iq[['total_cases']],ceil_train_pred_iq))
print(mean_absolute_error(data_iq[['total_cases']],ceil_train_pred_iq))
print(mean_absolute_error(data_iq[['total_cases']],round_train_pred_iq))
print(sqrt_iq)

round_train_pred = round_train_pred_sj + round_train_pred_iq
sqrt = np.sqrt(metrics.mean_squared_error(data[['total_cases']],round_train_pred))
print(sqrt)

imputer = Imputer()
test_data_sj = test_data_sj[selected_f_sj]
test_imputed_sj = imputer.fit_transform(test_data_sj)

test_pred_sj = linreg_sj.predict(poly_sj.fit_transform(test_imputed_sj))

round_test_pred_sj =[]
for d in test_pred_sj:
    if(d<0):
        d=0

    else:
        d=round(d)
    round_test_pred_sj.append(int(d))

test_data_iq = test_data_iq[selected_f]
test_imputed_iq = imputer.fit_transform(test_data_iq)

test_pred_iq = linreg_iq.predict(poly_iq.fit_transform(test_imputed_iq))

round_test_pred_iq =[]
for d in test_pred_iq:
    if(d<0):
        d=0

    else:
        d=round(d)
    round_test_pred_iq.append(int(d))



round_test_pred =round_test_pred_sj+round_test_pred_iq
submission_data = submission_data.drop(['total_cases'], axis=1)
round_test_pred = pd.DataFrame(round_test_pred)

submission_data = pd.concat([submission_data, round_test_pred], axis=1, join_axes=[submission_data.index])

submission_data = submission_data.rename(columns={0:'total_cases'})

submission_data.to_csv("submission_5-22.csv",encoding='utf-8', index=False)
    
